In [1]:
import pandas as pd
import numpy as np

In [2]:
survey_client = pd.read_csv('Datasets/survey_client.csv', dtype={'Customer heading': int})
survey_client.drop('Unnamed: 0', axis=1, inplace=True)
survey_client['teacher'] = survey_client['Customer heading'].astype(str) + '_' + survey_client['Subject'] + '_' + survey_client['Course']
survey_client = survey_client.drop_duplicates()

In [18]:
survey_client

,Year,Course,Subject,# copies,Customer heading,teacher
0,2015,primary 1,natural sciences,17,136377,136377_natural sciences_primary 1
1,2015,primary 1,natural sciences,60,136384,136384_natural sciences_primary 1
2,2015,primary 1,natural sciences,72,136387,136387_natural sciences_primary 1
3,2015,primary 1,natural sciences,30,136393,136393_natural sciences_primary 1
4,2015,primary 1,natural sciences,52,136405,136405_natural sciences_primary 1
...,...,...,...,...,...,...
30915,2019,primary 3,music,51,212529,212529_music_primary 3
30916,2019,primary 1,spanish,111,212717,212717_spanish_primary 1
30917,2019,primary 2,spanish,114,212717,212717_spanish_primary 2
30918,2019,primary 6,spanish,166,212717,212717_spanish_primary 6


In [12]:
churn_targets = survey_client.pivot(index='teacher', columns='Year', values='# copies').fillna(0)

In [14]:
for year in [2015,2016,2017,2018]:
    colname = 'churn_' + str(year)
    churn_targets[colname] = churn_targets.apply(lambda row: 1 if (row[year] > 0) and (row[year+1] == 0) else 0, axis=1)

In [15]:
churn_targets

Year,2015,2016,2017,2018,2019,churn_2015,churn_2016,churn_2017,churn_2018
teacher,,,,,,,,,
136369_mathematics_primary 1,136.0,0.0,140.0,0.0,140.0,1,0,1,0
136369_mathematics_primary 2,0.0,136.0,135.0,0.0,139.0,0,0,1,0
136369_mathematics_primary 3,144.0,0.0,138.0,0.0,134.0,1,0,1,0
136369_mathematics_primary 4,140.0,144.0,0.0,135.0,0.0,0,1,0,1
136369_mathematics_primary 5,148.0,136.0,140.0,140.0,0.0,0,0,0,1
...,...,...,...,...,...,...,...,...,...
212719_spanish_primary 2,0.0,0.0,120.0,0.0,0.0,0,0,1,0
212719_spanish_primary 3,0.0,0.0,109.0,0.0,0.0,0,0,1,0
212719_spanish_primary 4,0.0,0.0,154.0,109.0,0.0,0,0,0,1


In [16]:
def assign_churn_target(row):
    if row.Year == 2019:
        return np.nan
    else:
        colname = 'churn_' + str(row.Year)
        return churn_targets.loc[row.teacher, colname]

In [19]:
survey_client['churn'] = survey_client.apply(lambda row: assign_churn_target(row), axis=1)

In [32]:
survey_client.to_csv('survey_client_with_churn.csv', index=False)

In [26]:
survey_client[(survey_client.Subject == 'mathematics') & (survey_client.Course == 'primary 4')]

,Year,Course,Subject,# copies,Customer heading,teacher,churn
2631,2015,primary 3,mathematics,144,136369,136369_mathematics_primary 3,1.0
2632,2015,primary 3,mathematics,120,136370,136370_mathematics_primary 3,0.0
2633,2015,primary 3,mathematics,121,136372,136372_mathematics_primary 3,0.0
2634,2015,primary 3,mathematics,31,136377,136377_mathematics_primary 3,0.0
2635,2015,primary 3,mathematics,57,136384,136384_mathematics_primary 3,1.0
...,...,...,...,...,...,...,...
30767,2019,primary 3,mathematics,0,210087,210087_mathematics_primary 3,NaN
30796,2019,primary 3,mathematics,77,210302,210302_mathematics_primary 3,NaN
30835,2019,primary 3,mathematics,57,210342,210342_mathematics_primary 3,NaN
30860,2019,primary 3,mathematics,83,210414,210414_mathematics_primary 3,NaN
